In [1]:
import pandas as pd
import useful_functions as  use
import pyodbc

In [77]:
username = 'vigrose'
password = 'Ravenclaw~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(w)

In [2]:
ppd = use.get_ppd()

C:/Users/vigrose/Data/PPD\ppd_data_20210731.csv


C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (13,14,20,21,49,56,57,72) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
ppd.groupby('NO_CONTACT_IND').count()

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
NO_CONTACT_IND,,,,,,,,,,,,,,,,,,,,,
N,44193,44193,44193,43580,44193,44193,44192,37770,1565,6613,...,32756,32777,32315,32490,13331,11490,13330,1669,1398,1669


In [71]:
username = 'vigrose'
password = 'Ravenclaw~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password)
AMAEDW = pyodbc.connect(w)
ppd = ppd.fillna('None')
mes = tuple(ppd[ppd.NO_CONTACT_IND!='None'].ME)
ME_QUERY = \
        f"""
        SELECT DISTINCT
        P.PARTY_ID,
        P.KEY_VAL AS ME
        FROM
        AMAEDW.PARTY_KEY P
        WHERE
        P.KEY_TYPE_ID = 18
        AND
        P.ACTIVE_IND = 'Y'
        AND 
        P.KEY_VAL in {mes}
        """
ME = pd.read_sql(con=AMAEDW, sql=ME_QUERY)
party_ids = tuple(ME.PARTY_ID)
sql_query = \
    f'''
    SELECT DISTINCT
    N.PHONE_NBR,
    P.PARTY_ID,
    FROM
    AMAEDW.PHONE_NBR N, AMAEDW.PARTY_PHONE P, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    P.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    N.PHONE_ID = P.PHONE_ID
    AND
    P.SRC_END_DT IS NULL
    AND
    C.PURPOSE_CAT_CD='P'
    AND
    C.PURPOSE_USG_CD = 'PV'
    AND
    P.PARTY_ID in
    '''
phones_2 = pd.read_sql(con=AMAEDW, sql=sql_query)

In [72]:
mes = tuple(ppd[ppd.NO_CONTACT_IND!='None'].ME)

In [73]:
ME_QUERY = \
        f"""
        SELECT DISTINCT
        P.PARTY_ID,
        P.KEY_VAL AS ME
        FROM
        AMAEDW.PARTY_KEY P
        WHERE
        P.KEY_TYPE_ID = 18
        AND
        P.ACTIVE_IND = 'Y'
        AND 
        P.KEY_VAL in {mes}
        """
ME = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [74]:
party_ids = tuple(ME.PARTY_ID)

In [78]:
sql_query = \
    f'''
    SELECT DISTINCT
    N.AREA_CD,
    N.EXCHANGE,
    N.PHONE_NBR,
    P.PARTY_ID,
    CC.DESC,
    C.PURPOSE_USG_CD,
    C.PURPOSE_TYPE_DESC
    FROM
    AMAEDW.PHONE_NBR N, AMAEDW.PARTY_PHONE P, AMAEDW.CAT_CD CC, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    P.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    N.PHONE_ID = P.PHONE_ID
    AND
    P.SRC_END_DT IS NULL
    AND
    CC.CAT_CD_ID = P.CAT_CD_ID
    AND 
    P.PARTY_ID in {party_ids}
    AND
    C.PURPOSE_CAT_CD='P'
    AND
    C.PURPOSE_USG_CD = 'PV';
    '''
phones_2 = pd.read_sql(con=AMAEDW, sql=sql_query)

In [80]:
phones_2.drop_duplicates('PARTY_ID')

,AREA_CD,EXCHANGE,PHONE_NBR,PARTY_ID,DESC,PURPOSE_USG_CD,PURPOSE_TYPE_DESC
0,513,872,4222,1798033,TELEPHONE SURVEY DATA,PV,Office
1,205,836,8691,1798154,Physician Census,PV,Office
2,206,842,3222,1798161,HSG VHCP DATA,PV,Office
3,205,975,7891,1798398,EIB Communication for batch only,PV,Office
4,205,345,1520,1798456,Other Source,PV,Office
...,...,...,...,...,...,...,...
30035,412,864,6600,12413113,HSG WEBVRTR DATA,PV,Office
30036,714,916,5210,12609060,HSG WEBVRTR DATA,PV,Office
30037,870,347,2534,12856418,HSG WEBVRTR DATA,PV,Office
30038,928,853,0344,13043094,HSG WEBVRTR DATA,PV,Office


In [81]:
for row in PHONES.itertuples():
    phone = f'{row.AREA_CD}{row.EXCHANGE}{row.PHONE_NBR}'
    

,AREA_CD,EXCHANGE,PHONE_NBR,PARTY_ID,DESC
0,513,872,4222,1798033,TELEPHONE SURVEY DATA
1,205,836,8691,1798154,Physician Census
2,206,842,3222,1798161,HSG VHCP DATA
3,205,975,7891,1798398,EIB Communication for batch only
4,205,345,1520,1798456,Other Source
...,...,...,...,...,...
30007,412,864,6600,12413113,HSG WEBVRTR DATA
30008,714,916,5210,12609060,HSG WEBVRTR DATA
30009,870,347,2534,12856418,HSG WEBVRTR DATA
30010,928,853,0344,13043094,HSG WEBVRTR DATA


In [26]:
ME[ME.PARTY_ID.isin(PHONES.PARTY_ID)]

,PARTY_ID,ME
1,1892054,00102801644
2,1888898,00102811020
3,1892074,00102811747
5,1888753,00102720041
8,1891802,00102730755
...,...,...
43997,12413113,06702140011
44000,12609060,00576151683
44005,12856418,00576141173
44007,13043094,01676150885


In [60]:
ppd = pd.merge(ppd, MES, on='ME')

In [62]:
#NO PPD PHONE BUT PHONES WITHOUT ENDDATES , NO USAGE?
no_phone = tuple(ppd[(ppd.TELEPHONE_NUMBER=='None')&(ppd.TOP_CD==20)&(ppd.NO_CONTACT_IND=='None')].PARTY_ID)

In [59]:
ME_QUERY = \
        f"""
        SELECT DISTINCT
        P.PARTY_ID,
        P.KEY_VAL AS ME
        FROM
        AMAEDW.PARTY_KEY P
        WHERE
        P.KEY_TYPE_ID = 18
        AND
        P.ACTIVE_IND = 'Y'
        """
MES = pd.read_sql(con=AMAEDW, sql=ME_QUERY)

In [54]:
these_party_ids = tuple(MESS.PARTY_ID)
sql_query = \
    f'''
    SELECT DISTINCT
    N.AREA_CD,
    N.EXCHANGE,
    N.PHONE_NBR,
    P.PARTY_ID,
    CC.DESC AS CATEGORY_DESC,
    C.PURPOSE_USG_CD,
    C.PURPOSE_TYPE_DESC,
    C.PURPOSE_USG_DESC
    FROM
    AMAEDW.PHONE_NBR N, AMAEDW.PARTY_PHONE P, AMAEDW.CAT_CD CC, AMAEDW.CONT_PURPOSE_TYPE C
    WHERE
    P.PURPOSE_TYPE_ID=C.PURPOSE_TYPE_ID
    AND
    N.PHONE_ID = P.PHONE_ID
    AND
    P.SRC_END_DT IS NULL
    AND
    CC.CAT_CD_ID = P.CAT_CD_ID
    AND 
    C.PURPOSE_CAT_CD='P'
    AND
    C.PURPOSE_USG_CD = 'PV' ;
    '''
other_phones = pd.read_sql(con=AMAEDW, sql=sql_query)

In [55]:
actually_phones[(actually_phones.DESC!='HSG WEBVRTR DATA')]

,AREA_CD,EXCHANGE,PHONE_NBR,PARTY_ID,DESC,PURPOSE_USG_CD,PURPOSE_TYPE_DESC,PURPOSE_USG_DESC
1,412,780,8849,1798288,Account Management Customer Communication,AMC,Cellular,AMC Voice ...
2,412,780,8849,1798288,Account Management Customer Communication,None,Cellular,No Usage
3,412,780,8849,1798288,Account Management Customer Communication,PV,Cellular,Preferred Voice
4,706,248,9971,1798298,Phone Call,None,Cellular,No Usage
5,903,938,9255,1798440,External E-mail,None,Home,No Usage
...,...,...,...,...,...,...,...,...
41850,915,215,4600,15942455,Phone Call,None,Home,No Usage
41851,608,358,7735,15942455,Phone Call,None,Cellular,No Usage
41852,817,832,5533,15949492,Phone Call,None,Cellular,No Usage
41853,818,521,1756,15953861,Phone Call,None,Cellular,No Usage


In [63]:
other_phones[(other_phones.CATEGORY_DESC!='HSG WEBVRTR DATA')&(other_phones.PARTY_ID.isin(no_phone))]

,AREA_CD,EXCHANGE,PHONE_NBR,PARTY_ID,CATEGORY_DESC,PURPOSE_USG_CD,PURPOSE_TYPE_DESC,PURPOSE_USG_DESC
289,412,780,8849,1798288,Account Management Customer Communication,AMC,Cellular,AMC Voice ...
290,412,780,8849,1798288,Account Management Customer Communication,None,Cellular,No Usage
309,706,248,9971,1798298,Phone Call,None,Cellular,No Usage
331,203,506,3384,1798307,Account Management Customer Communication,None,Cellular,No Usage
335,513,475,8990,1798307,Account Management Customer Communication,AMC,Office,AMC Voice ...
...,...,...,...,...,...,...,...,...
1828197,215,791,0408,16001050,Phone Call,None,Cellular,No Usage
1828526,469,286,8063,16025291,Phone Call,None,Home,No Usage
1829305,409,651,3476,16091459,Phone Call,None,Cellular,No Usage
1830272,570,815,4841,16190712,Phone Call,None,Home,No Usage


In [65]:
len(no_phone)

117209

In [66]:
len(mes)

44037

In [12]:
PPD.groupby('ADDRESS_TYPE').count()

,ME,RECORD_ID,UPDATE_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,MAILING_LINE_2,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
ADDRESS_TYPE,,,,,,,,,,,,,,,,,,,,,
1.0,502102,502102,502102,502102,502102,502102,502102,502102,502102,502102,...,502102,502102,502102,502102,502102,502102,502102,502102,502102,502102
2.0,745798,745798,745798,745798,745798,745798,745798,745798,745798,745798,...,745798,745798,745798,745798,745798,745798,745798,745798,745798,745798
3.0,3672,3672,3672,3672,3672,3672,3672,3672,3672,3672,...,3672,3672,3672,3672,3672,3672,3672,3672,3672,3672
None,18842,18842,18842,18842,18842,18842,18842,18842,18842,18842,...,18842,18842,18842,18842,18842,18842,18842,18842,18842,18842


In [11]:
PPD = PPD.fillna('None')

In [8]:
ppd.groupby('PRESUMED_DEAD_FLAG').count()

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
PRESUMED_DEAD_FLAG,,,,,,,,,,,,,,,,,,,,,
D,6197,6197,6197,5863,6197,6196,6197,4894,273,647,...,2210,2235,2077,2116,2994,2550,2994,242,193,242


In [9]:
PPD = ppd[(ppd.ADDRESS_UNDELIVERABLE_FLAG!='U')&(ppd.PRESUMED_DEAD_FLAG!='D')]

In [21]:
ppd[(ppd.TOP_CD==20)&(ppd.PRIM_SPEC_CD!='US')&(ppd.PE_CD!=110)].count()['POLO_ZIP']

580803

In [23]:
ppd[(ppd.TOP_CD==20)&(ppd.PRIM_SPEC_CD!='US')&(ppd.PE_CD!=110)]

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
0,00102000013,A,1,2.0,MICHAEL J STANFORD MD,STANFORD,MICHAEL,JAMES,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00102000030,A,1,1.0,MARK T JACKSON MD,JACKSON,MARK,THOMAS,NaN,NaN,...,NC,28805,1262.0,C082,NaN,NaN,NaN,NaN,NaN,NaN
3,00102000048,A,1,2.0,NADHIWAN S MAISIAK MD,MAISIAK,NADHIWAN,SAKARIN,NaN,NaN,...,AZ,85375,5282.0,C045,NaN,NaN,NaN,NaN,NaN,NaN
4,00102000056,A,1,2.0,RAJIV B GALA MD,GALA,RAJIV,BABULAL,NaN,NaN,...,LA,70115,6949.0,C003,NaN,NaN,NaN,NaN,NaN,NaN
5,00102000064,A,1,2.0,CAROL L ADAMS MD,ADAMS,CAROL,LYNN,NaN,NaN,...,AL,35504,2516.0,C009,DANIELS,LYNN,CAROL,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1301983,96501930013,A,1,2.0,ANANT K SHAH MD,SHAH,ANANT,KUMAR,NaN,NaN,...,FL,33542,3501.0,C069,SHAH,M,ANANTKUMAR,NaN,NaN,NaN
1301984,96501930021,A,1,2.0,RAJIV JETLY MD,JETLY,RAJIV,NaN,NaN,NaN,...,AZ,86314,1222.0,H079,NaN,NaN,NaN,NaN,NaN,NaN
1301985,96501940019,A,1,1.0,MUKUL GUPTA MD,GUPTA,MUKUL,NaN,NaN,NaN,...,CA,93105,4230.0,C013,NaN,NaN,NaN,NaN,NaN,NaN
1301986,96501950014,A,1,2.0,DINESH BHASKARAN MD,BHASKARAN,DINESH,NaN,NaN,NaN,...,PA,15009,9727.0,R001,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
(580803/640553)*0.7774

0.7048850792986685

In [ ]:
us
110

In [37]:
suffix_ppd = ppd.dropna(subset = ['SUFFIX'])

In [40]:
suffix_ppd['SUFFIX_DESC'] = [x.split(' ')[-2] for x in suffix_ppd.MAILING_NAME]

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
ppd[ppd.SUFFIX.isna].MAILING_NAME

TypeError: isna() takes 1 positional argument but 2 were given

In [47]:
suffix_ppd.groupby(['SUFFIX','SUFFIX_DESC','GENDER']).count()[['ME']]

ME
SUFFIX SUFFIX_DESC GENDER       
1.0    SR          1.0       440
                   2.0        23
2.0    JR          1.0     25944
                   2.0        38
3.0    II          1.0      3303
                   2.0        18
4.0    III         1.0      7812
                   2.0        15
5.0    IV          1.0      1118
                   2.0         6
6.0    V           1.0        85
                   2.0         4
7.0    VI          1.0         9

In [46]:
ppd.groupby('GROUP').count()

,ME,RECORD_ID,UPDATE_TYPE,ADDRESS_TYPE,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,MAILING_LINE_1,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
GROUP,,,,,,,,,,,,,,,,,,,,,
0.0,8837,8837,8837,3770,8837,8837,8837,6700,65,6478,...,1421,2592,1387,1402,1844,1499,1844,98,72,98
1.0,3880,3880,3880,3683,3880,3880,3880,3513,166,715,...,2675,2675,2539,2582,1046,951,1046,101,88,101
2.0,21451,21451,21451,20796,21451,21451,21450,17916,913,3448,...,16064,16064,15673,15812,5468,4891,5468,604,504,604
3.0,46582,46582,46582,45270,46582,46582,46582,40665,2236,7465,...,34446,34447,33618,33888,13100,11326,13100,1431,1180,1431
4.0,79391,79391,79391,78170,79391,79391,79390,68772,3827,12209,...,60420,60421,59393,59745,20464,18014,20464,2246,1911,2246
5.0,193907,193907,193907,191002,193907,193902,193904,162969,7614,32415,...,145401,145400,143250,143977,48370,41508,48371,5392,4384,5392
6.0,946073,946073,946073,936961,946073,946048,946057,741068,23949,165246,...,711519,711516,703392,706166,205690,169455,205690,23969,18788,23970
